# Carga datos places desde el bucket de AWS

In [1]:
"""   Librerías   """
import pandas as pd # Manipulación y EDA
import matplotlib.pyplot as plt # Gráficos
import seaborn as sns # Gráficos
from dataAccess import aws_df_from_S3_csv, aws_subir_archivo_s3 # AWS

In [2]:
dataset = awsFunctions.aws_df_from_S3_csv("Biel/colegios_madrid.csv")
dataset = dataset.drop(['Unnamed: 0'], axis = 1)
dataset.head()

,name,address,price,review,description,type
0,Virgen Del Cerro,"Calle Mohernando 1, 28038, Madrid, Madrid",Menos de 100€/mes,4.3 estrellas,Colegio Público de Educación Infantil y Primaria,Publico
1,Miguel De Cervantes,"Avenida Villaviciosa s/n, 28922, Alcorcón, Madrid",NaN,4.3 estrellas,"Colegio Público de Educación Infantil, Primari...",Publico
2,República Del Paraguay,"Calle Manresa 60, 28034, Madrid, Madrid",Menos de 100€/mes,4.7 estrellas,Colegio Público de Educación Infantil y Primaria,Publico
3,San José De Valderas,"Avenida de los Derechos Humanos 6, 28924, Alco...",Menos de 100€/mes,4.1 estrellas,Colegio Público de Educación Infantil y Primaria,Publico
4,Filosofo Séneca,"Calle Santa Adela 21, 28033, Madrid, Madrid",Menos de 100€/mes,4.1 estrellas,Colegio Público de Educación Infantil y Primaria,Publico


In [3]:
# Número de registros que contiene el dataset
print("El dataset contiene un total de {} establecimientos.\n".format(len(dataset)))

El dataset contiene un total de 1786 establecimientos.



In [4]:
# Duplicados  
print(dataset.duplicated().sum())
# Nulos
print(dataset.isnull().sum())

0
name           0
address        0
price          2
review         0
description    0
type           0
dtype: int64


In [10]:
df_clean = df_clean.reset_index().rename(columns={"index": "_id"})
df_clean.head()

,_id,name,address,price,review,description,type
0,0,Virgen Del Cerro,"Calle Mohernando 1, 28038, Madrid, Madrid",Menos de 100€/mes,4.3 estrellas,Colegio Público de Educación Infantil y Primaria,Publico
1,1,República Del Paraguay,"Calle Manresa 60, 28034, Madrid, Madrid",Menos de 100€/mes,4.7 estrellas,Colegio Público de Educación Infantil y Primaria,Publico
2,2,San José De Valderas,"Avenida de los Derechos Humanos 6, 28924, Alco...",Menos de 100€/mes,4.1 estrellas,Colegio Público de Educación Infantil y Primaria,Publico
3,3,Filosofo Séneca,"Calle Santa Adela 21, 28033, Madrid, Madrid",Menos de 100€/mes,4.1 estrellas,Colegio Público de Educación Infantil y Primaria,Publico
4,4,Beatriz Galindo,"Calle de Goya 10, 28001, Madrid, Madrid",Menos de 100€/mes,3.5 estrellas,"Instituto Público de Educación Secundaria, Bac...",Publico


## Duplicados por Name, Address, Price

Se mantendría el registro con la descripción más larga, aporta más información. Como no se va a utilizar la descripción para hacer feature engineering, borramos  duplicados alegremente.

In [12]:
df_clean = df_clean.drop_duplicates(subset=["name","address","price"])

In [13]:
# Número de registros que contiene el dataset
print("El dataset contiene un total de {} establecimientos.\n".format(len(dataset)))

El dataset contiene un total de 1786 establecimientos.



## Google Maps API

Se asocian las coordenadas de cada colegio mediante Places API.

In [15]:
import requests as rq
import json
from types import SimpleNamespace
import time

In [53]:
def text_search(key,address):
    response = rq.get(url="https://maps.googleapis.com/maps/api/place/textsearch/json", 
                  params={
                      "query": address,
                      "key": key,
                      })
    return json.loads(response.text, object_hook=lambda d: SimpleNamespace(**d))

def peticion_individual(key,address):
    time.sleep(3)
    response = text_search(key,address)
    time.sleep(3)
    element = response.results[0]
    result = str(element.geometry.location.lat)+","+str(element.geometry.location.lng)
    return result

In [17]:
# Peticiones
KEY = "" # Biel
KEY = "" # Arnau
df_clean["query"] = "school "+df_clean["name"]+", "+df_clean["address"]

C:\Users\arni_\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [19]:
# Primer intento
coord = dict()
id_error = list()

for elem in df_clean._id:
    try:
        coord[elem] = peticion_individual(KEY,df_clean['query'][elem])
    except:
        id_error.append(elem)

In [20]:
print(id_error)

[7, 8, 9, 10, 11, 12, 13, 21, 22, 23, 24, 25, 26, 34, 35, 36, 37, 38, 39, 48, 49, 50, 51, 52, 53, 62, 63, 64, 65, 66, 67, 75, 76, 77, 78, 79, 80, 88, 89, 90, 91, 92, 93, 94, 102, 103, 104, 105, 106, 107, 109, 117, 118, 119, 120, 121, 129, 130, 131, 132, 133, 134, 135, 144, 145, 146, 147, 148, 149, 157, 158, 159, 160, 161, 162, 170, 171, 172, 174, 175, 176, 184, 185, 186, 187, 188, 189, 190, 198, 199, 200, 201, 202, 203, 211, 212, 213, 214, 215, 216, 219, 226, 227, 228, 229, 230, 231, 232, 240, 241, 242, 243, 244, 252, 253, 254, 255, 256, 257, 265, 266, 267, 268, 269, 270, 278, 279, 280, 281, 282, 283, 291, 292, 293, 294, 295, 296, 297, 305, 306, 307, 308, 309, 310, 312, 320, 321, 322, 323, 324, 325, 326, 334, 336, 337, 338, 339, 340, 348, 349, 350, 351, 352, 353, 354, 362, 363, 364, 365, 366, 367, 375, 376, 377, 378, 379, 380, 388, 389, 390, 391, 392, 393, 394, 395, 403, 404, 405, 406, 407, 415, 416, 417, 418, 419, 420, 421, 422, 430, 431, 432, 433, 434, 435, 437, 441, 444, 445, 446, 4

In [21]:
# Segundo intento
id_error_2 = list()

for elem in id_error:
    try:
        coord[elem] = peticion_individual(KEY,df_clean['query'][elem])
    except:
        id_error_2.append(elem)

In [22]:
print(id_error_2)

[7, 8, 9, 22, 25, 26, 34, 35, 36, 52, 53, 62, 63, 64, 65, 66, 88, 89, 90, 91, 92, 93, 109, 117, 118, 119, 120, 121, 129, 145, 146, 147, 148, 149, 157, 172, 174, 175, 176, 184, 185, 186, 201, 202, 203, 211, 212, 213, 229, 230, 231, 232, 240, 241, 242, 257, 265, 266, 267, 268, 269, 291, 292, 293, 294, 295, 296, 297, 320, 321, 322, 323, 324, 325, 326, 349, 350, 351, 352, 353, 354, 376, 377, 378, 379, 380, 388, 389, 404, 405, 406, 407, 415, 416, 417, 432, 433, 434, 435, 437, 441, 458, 459, 460, 461, 462, 470, 485, 486, 487, 488, 489, 497, 498, 513, 514, 515, 516, 524, 525, 541, 542, 543, 544, 552, 567, 568, 569, 571, 579, 580, 589, 594, 595, 596, 597, 598, 621, 622, 623, 624, 625, 647, 649, 650, 651, 652, 655, 675, 676, 677, 678, 679, 687, 702, 703, 704, 705, 706, 714, 715, 730, 731, 732, 733, 741, 742, 757, 758, 759, 760, 768, 769, 770, 785, 786, 787, 788, 797, 812, 813, 814, 815, 816, 824, 839, 840, 841, 842, 850, 851, 852, 853, 868, 869, 877, 878, 879, 894, 895, 896, 904, 905, 906, 907,

In [23]:
# Tercer intento
id_error_3 = list()

for elem in id_error_2:
    try:
        coord[elem] = peticion_individual(KEY,df_clean['query'][elem])
    except:
        id_error_3.append(elem)

In [24]:
print(id_error_3)

[26, 34, 35, 36, 52, 53, 90, 92, 93, 109, 117, 118, 148, 149, 157, 172, 174, 175, 184, 212, 229, 230, 231, 267, 268, 269, 291, 292, 293, 324, 325, 326, 349, 350, 351, 380, 388, 389, 404, 405, 406, 437, 441, 458, 459, 460, 461, 497, 498, 513, 514, 515, 516, 524, 568, 569, 571, 579, 580, 589, 623, 624, 625, 647, 649, 650, 651, 687, 702, 703, 704, 705, 706, 742, 757, 758, 759, 760, 768, 769, 770, 814, 815, 816, 824, 839, 868, 869, 877, 878, 879, 922, 924, 932, 933, 934, 949, 950, 961, 987, 988, 989, 1004, 1012, 1013, 1042, 1043, 1044, 1066, 1067, 1068, 1097, 1098, 1106, 1121, 1122, 1123, 1124, 1125, 1136, 1151, 1154, 1162, 1175, 1177, 1178, 1215, 1216, 1217, 1218, 1233, 1234, 1241, 1270, 1272, 1287, 1288, 1296, 1306, 1325, 1326, 1328, 1351, 1352, 1381, 1404, 1405, 1406, 1407, 1408, 1409, 1445, 1460, 1461, 1462, 1463, 1471, 1504, 1513, 1514, 1515, 1516, 1517, 1525, 1569, 1570, 1578, 1579, 1580, 1581, 1598, 1634, 1635, 1636, 1637, 1652, 1660, 1689, 1690, 1691, 1692, 1715, 1716, 1721, 1736, 

In [25]:
# Cuarto intento
id_error_4 = list()

for elem in id_error_3:
    try:
        coord[elem] = peticion_individual(KEY,df_clean['query'][elem])
    except:
        id_error_4.append(elem)

In [26]:
print(id_error_4)

[52, 53, 90, 92, 93, 174, 175, 184, 212, 229, 230, 231, 325, 326, 349, 350, 351, 380, 441, 458, 459, 460, 461, 497, 498, 513, 579, 580, 589, 623, 624, 625, 704, 705, 706, 742, 757, 758, 759, 839, 868, 869, 877, 878, 950, 961, 987, 988, 989, 1004, 1012, 1013, 1098, 1106, 1121, 1122, 1123, 1136, 1151, 1175, 1177, 1178, 1215, 1216, 1217, 1218, 1233, 1241, 1306, 1325, 1326, 1328, 1351, 1352, 1404, 1445, 1460, 1461, 1462, 1463, 1471, 1504, 1570, 1578, 1579, 1580, 1581, 1598, 1690, 1691, 1692, 1715, 1716, 1721, 1736, 1743, 1745]


In [34]:
# Quinto intento
id_error_5 = list()

for elem in id_error_4:
    try:
        coord[elem] = peticion_individual(KEY,df_clean['query'][elem])
    except:
        id_error_5.append(elem)

In [39]:
print(id_error_5,len(df_clean))

[184, 212, 229, 230, 231, 325, 326, 349, 441, 461, 497, 498, 513, 589, 706, 742, 757, 758, 759, 839, 868, 989, 1004, 1012, 1013, 1098, 1106, 1136, 1151, 1175, 1177, 1178, 1215, 1216, 1217, 1218, 1233, 1241, 1306, 1404, 1445, 1460, 1462, 1463, 1471, 1504, 1690, 1691, 1692, 1715, 1716, 1721, 1736, 1743] 1772


In [40]:
# Sexto intento
id_error_6 = list()

for elem in id_error_5:
    try:
        coord[elem] = peticion_individual(KEY,df_clean['query'][elem])
    except:
        id_error_6.append(elem)

In [41]:
print(id_error_6,len(df_clean))

[229, 349, 441, 461, 497, 498, 513, 589, 989, 1004, 1012, 1013, 1098, 1106, 1136, 1151, 1175, 1177, 1217, 1218, 1233, 1241, 1306, 1404, 1445, 1463, 1504, 1715, 1716, 1721, 1736, 1743] 1772


In [42]:
# Séptimo intento
id_error_7 = list()

for elem in id_error_6:
    try:
        coord[elem] = peticion_individual(KEY,df_clean['query'][elem])
    except:
        id_error_7.append(elem)

In [43]:
print(id_error_7,len(df_clean))

[229, 441, 589, 989, 1004, 1012, 1013, 1098, 1106, 1136, 1151, 1175, 1177, 1241, 1306, 1404, 1445, 1463, 1504, 1721, 1736, 1743] 1772


In [44]:
# Oct intento
id_error_8 = list()

for elem in id_error_7:
    try:
        coord[elem] = peticion_individual(KEY,df_clean['query'][elem])
    except:
        id_error_8.append(elem)

In [45]:
print(id_error_8,len(df_clean))

[229, 441, 589, 1098, 1106, 1136, 1151, 1175, 1177, 1241, 1306, 1404, 1504, 1721, 1736, 1743] 1772


In [59]:
# Intentos restantes
id_error_12 = list()

for elem in id_error_11:
    try:
        coord[elem] = peticion_individual(KEY,df_clean['query'][elem])
    except:
        id_error_12.append(elem)

In [60]:
print(len(id_error_12),len(coord),len(df_clean))

14 1758 1772


In [61]:
# convertir a df y mergear por id
d = {'_id' : coord.keys(), 'coord' : coord.values()}
coord_df = pd.DataFrame(data = d)
coord_df

,_id,coord
0,0,"40.3960957,-3.6490087"
1,1,"40.492446,-3.695208"
2,2,"40.3534264,-3.8153475"
3,3,"40.4783861,-3.6502275"
4,4,"40.4250762,-3.6864769"
...,...,...
1753,1013,"40.3741678,-3.7531309"
1754,1445,"40.4736132,-3.720858"
1755,1463,"40.4190208,-3.6117053"
1756,1098,"40.35024809999999,-3.8119025"


In [65]:
df_clean_coord = df_clean.merge(coord_df, on = '_id')

In [66]:
df_clean_coord.head()

,_id,name,address,price,review,description,type,query,coord
0,0,Virgen Del Cerro,"Calle Mohernando 1, 28038, Madrid, Madrid",Menos de 100€/mes,4.3 estrellas,Colegio Público de Educación Infantil y Primaria,Publico,"school Virgen Del Cerro, Calle Mohernando 1, 2...","40.3960957,-3.6490087"
1,1,República Del Paraguay,"Calle Manresa 60, 28034, Madrid, Madrid",Menos de 100€/mes,4.7 estrellas,Colegio Público de Educación Infantil y Primaria,Publico,"school República Del Paraguay, Calle Manresa 6...","40.492446,-3.695208"
2,2,San José De Valderas,"Avenida de los Derechos Humanos 6, 28924, Alco...",Menos de 100€/mes,4.1 estrellas,Colegio Público de Educación Infantil y Primaria,Publico,"school San José De Valderas, Avenida de los De...","40.3534264,-3.8153475"
3,3,Filosofo Séneca,"Calle Santa Adela 21, 28033, Madrid, Madrid",Menos de 100€/mes,4.1 estrellas,Colegio Público de Educación Infantil y Primaria,Publico,"school Filosofo Séneca, Calle Santa Adela 21, ...","40.4783861,-3.6502275"
4,4,Beatriz Galindo,"Calle de Goya 10, 28001, Madrid, Madrid",Menos de 100€/mes,3.5 estrellas,"Instituto Público de Educación Secundaria, Bac...",Publico,"school Beatriz Galindo, Calle de Goya 10, 2800...","40.4250762,-3.6864769"


In [67]:
df_clean_coord.isnull().sum()

_id            0
name           0
address        0
price          0
review         0
description    0
type           0
query          0
coord          0
dtype: int64

In [68]:
coords = df_clean_coord["coord"].str.split(",",expand=True)
coords.columns = ['latitude', 'longitude']
df_final = pd.concat([df_clean_coord, coords], axis=1)
df_final.head()

,_id,name,address,price,review,description,type,query,coord,latitude,longitude
0,0,Virgen Del Cerro,"Calle Mohernando 1, 28038, Madrid, Madrid",Menos de 100€/mes,4.3 estrellas,Colegio Público de Educación Infantil y Primaria,Publico,"school Virgen Del Cerro, Calle Mohernando 1, 2...","40.3960957,-3.6490087",40.3960957,-3.6490087
1,1,República Del Paraguay,"Calle Manresa 60, 28034, Madrid, Madrid",Menos de 100€/mes,4.7 estrellas,Colegio Público de Educación Infantil y Primaria,Publico,"school República Del Paraguay, Calle Manresa 6...","40.492446,-3.695208",40.492446,-3.695208
2,2,San José De Valderas,"Avenida de los Derechos Humanos 6, 28924, Alco...",Menos de 100€/mes,4.1 estrellas,Colegio Público de Educación Infantil y Primaria,Publico,"school San José De Valderas, Avenida de los De...","40.3534264,-3.8153475",40.3534264,-3.8153475
3,3,Filosofo Séneca,"Calle Santa Adela 21, 28033, Madrid, Madrid",Menos de 100€/mes,4.1 estrellas,Colegio Público de Educación Infantil y Primaria,Publico,"school Filosofo Séneca, Calle Santa Adela 21, ...","40.4783861,-3.6502275",40.4783861,-3.6502275
4,4,Beatriz Galindo,"Calle de Goya 10, 28001, Madrid, Madrid",Menos de 100€/mes,3.5 estrellas,"Instituto Público de Educación Secundaria, Bac...",Publico,"school Beatriz Galindo, Calle de Goya 10, 2800...","40.4250762,-3.6864769",40.4250762,-3.6864769


In [70]:
df_final.to_csv("datos_colegios_clean.csv", index = False)

# Subir los datos a S3

In [71]:
# El fichero a subir tiene que estar guardado en local
aws_subir_archivo_s3("data2gether-playground","datos_colegios_clean.csv","Arnau") # Si funciona devuelve true

True